In [2]:
import os
import rasterio
import glob
import numpy as np

In [3]:
# Go to raster directory
raster_dir = '../cities/sevilla/rasters'
os.chdir(raster_dir)

In [4]:
# List all .tif files
list = glob.glob('*.tif')
list

['SVF.tif',
 'average_blue.tif',
 'MDS.tif',
 'average_red.tif',
 'LST.tif',
 'average_green.tif',
 'GLI.tif',
 'average_nir.tif',
 'average_swir2.tif',
 'NBAI.tif',
 'average_swir1.tif',
 'NDTI.tif',
 'MDT.tif']

In [5]:
def normalize(input_path):
    output_path = input_path.replace('.tif', '_normalized.tif')
    
    with rasterio.open(input_path) as src:
        raster = src.read(1)
        
        # Ignore nodata values (if they exist)
        nodata = src.nodata
        mask = raster != nodata if nodata is not None else np.ones_like(raster, dtype=bool)
        
        # Calculate min and max (ignoring nodata values)
        valid_data = raster[mask]
        min_val = valid_data.min()
        max_val = valid_data.max()
        
        # Normalize raster values
        normalized_raster = np.zeros_like(raster, dtype=np.float32)
        normalized_raster[mask] = (valid_data - min_val) / (max_val - min_val)
        
        # Prepare output metadata
        meta = src.meta.copy()
        meta.update(dtype='float32', compress='lzw')
        
    # Write the normalized raster to a new file
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(normalized_raster, 1)
    
    print(f'Normalized raster saved to {output_path}')

In [6]:
mdt_raster_path = 'MDT.tif'
normalize(mdt_raster_path)

Normalized raster saved to MDT_normalized.tif


In [7]:
lst_raster_path = 'LST.tif'
normalize(lst_raster_path)

Normalized raster saved to LST_normalized.tif


In [5]:
mdt_raster_path = 'Zaragoza_ETRS89_MDT05.tif'
normalize(mdt_raster_path)

Normalized raster saved to Zaragoza_ETRS89_MDT05_normalized.tif


In [16]:
swir2_raster_path = 'Zaragoza_ETRS89_SWIR2.tif'
normalize(swir2_raster_path)

Normalized raster saved to Zaragoza_ETRS89_SWIR2_normalized.tif


In [5]:
mdt_raster_path = 'Zaragoza_ETRS89_LST_LC09_20230815.tif'
normalize(mdt_raster_path)

Normalized raster saved to Zaragoza_ETRS89_LST_LC09_20230815_normalized.tif


In [4]:
normalize('../../madrid/rasters/Madrid_ETR89_IMD.tif')

Normalized raster saved to ../../madrid/rasters/Madrid_ETR89_IMD_normalized.tif
